In [3]:
import pandas as pd

### Tratamento dos dados

In [56]:
file = open("Base_dados_golfe.txt", "r")

# Separa as linhas do arquivo
lines  = file.readlines()

# Pega o vetor de colunas
column_line = lines[1]
parts = column_line.split(": ")
column = list(parts[1].split("\n")[0].split("[")[1].split("]")[0].split(','))
column = [ele.strip().replace(" ", "") for ele in column]

# Pega o resto do conteúdo
content = []
content_lines = lines[3:-2]
for line in content_lines:
    parts = list(line.split("\n")[0].split("[")[1].split("]")[0].split(','))
    # Remove espaços
    parts = [ele.strip().replace(" ", "") for ele in parts]
    # Remove aspas
    parts = [ele.strip().replace("'", "") for ele in parts]
    content.append(parts)

# Transforma o conteúdo em um DataFrame
df = pd.DataFrame(content, columns=column)
# Remove 
df

,Tempo,Temperatura,Umidade,Vento,Jogar
0,Chuvoso,Quente,Alta,Fraco,Nao
1,Chuvoso,Quente,Alta,Forte,Nao
2,Nublado,Quente,Alta,Fraco,Sim
3,Ensolarado,Ameno,Alta,Fraco,Sim
4,Ensolarado,Frio,Normal,Fraco,Sim
5,Ensolarado,Frio,Normal,Forte,Nao
6,Nublado,Frio,Normal,Forte,Sim
7,Chuvoso,Ameno,Alta,Fraco,Nao
8,Chuvoso,Frio,Normal,Fraco,Sim
9,Ensolarado,Ameno,Normal,Fraco,Sim


### Cálculo das probabilidades

In [103]:
# Função para calcular as probabilidades

def calculateProbs(column_name, df_group):

    # Pega os valores únicos (ordenados ou padronizados)
    values = sorted(df_group[column_name].unique())

    # Cria dicionários seguros para manter a ordem do index igual para ambas as probabilidades
    df_yes = df_group[df_group['Jogar'] == 'Sim'].set_index(column_name)['Quantidade'].to_dict()
    df_not = df_group[df_group['Jogar'] == 'Nao'].set_index(column_name)['Quantidade'].to_dict()

    # Pega os valores de quantidade e transforma em uma lista
    not_qtds =  [df_not.get(val, 0) for val in values]
    yes_qtds =  [df_yes.get(val, 0) for val in values]

    not_total = sum(not_qtds)
    yes_total = sum(yes_qtds)

    prob_game = [val/yes_total for val in yes_qtds]
    prob_not_game = [val/not_total for val in not_qtds]

    # Imprime os valores de probabilidade:
    for i, val in enumerate(values):
        print(f'P(Jogo|{val}) = {prob_game[i]}\nP(Not_Jogo|{val}) = {prob_not_game[i]}\n')

    return prob_game, prob_not_game

#### 1. Jogo

In [104]:
# Calcula a probabilidade da condição jogo
df['Quantidade'] = 1
df_group = df.groupby("Jogar")['Quantidade'].sum().reset_index()
total = df_group['Quantidade'].sum()

# Probabilidade de ter NÃO jogo
not_qtd = df_group[df_group['Jogar'] == 'Nao']['Quantidade'].values[0]
P_NOT_GAME = not_qtd/total

# Probabilidade de ter jogo
yes_qtd = df_group[df_group['Jogar'] == 'Sim']['Quantidade'].values[0]
P_GAME = yes_qtd/total


print(f'P(Jogo) = {P_GAME}\nP(Not_Jogo) = {P_NOT_GAME}')
df_group


P(Jogo) = 0.6428571428571429
P(Not_Jogo) = 0.35714285714285715


,Jogar,Quantidade
0,Nao,5
1,Sim,9


#### 2. Tempo

In [107]:
df_group = df.groupby(["Jogar", "Tempo"])['Quantidade'].sum().reset_index()

# Adição de alpha para ter a opção de nublado em Jogar = Não

df_group['Quantidade'] = df_group['Quantidade'] + 1 # Incementa em 1 as outras opções
df_alpha = pd.DataFrame({
    'Jogar' : ['Nao'], 
    'Tempo' : ['Nublado'], 
    'Quantidade' : [1]
    })

df_group = pd.concat([df_group, df_alpha], ignore_index=True)

# CALCULA AS PROBABILIDADES
TEMPO_GAME, TEMPO_NOT_GAME = calculateProbs('Tempo', df_group)
df_group

P(Jogo|Chuvoso) = 0.25
P(Not_Jogo|Chuvoso) = 0.5

P(Jogo|Ensolarado) = 0.3333333333333333
P(Not_Jogo|Ensolarado) = 0.375

P(Jogo|Nublado) = 0.4166666666666667
P(Not_Jogo|Nublado) = 0.125



,Jogar,Tempo,Quantidade
0,Nao,Chuvoso,4
1,Nao,Ensolarado,3
2,Sim,Chuvoso,3
3,Sim,Ensolarado,4
4,Sim,Nublado,5
5,Nao,Nublado,1


#### 3. Temperatura

In [109]:
# Ordena o grupo
df_group = df.groupby(['Jogar', 'Temperatura'])['Quantidade'].sum().reset_index()

# CALCULA AS PROBABILIDADES
TEMPERATURA_GAME, TEMPERATURA_NOT_GAME = calculateProbs('Temperatura', df_group)
df_group



P(Jogo|Ameno) = 0.4444444444444444
P(Not_Jogo|Ameno) = 0.4

P(Jogo|Frio) = 0.3333333333333333
P(Not_Jogo|Frio) = 0.2

P(Jogo|Quente) = 0.2222222222222222
P(Not_Jogo|Quente) = 0.4



,Jogar,Temperatura,Quantidade
0,Nao,Ameno,2
1,Nao,Frio,1
2,Nao,Quente,2
3,Sim,Ameno,4
4,Sim,Frio,3
5,Sim,Quente,2


#### 4. Umidade

In [111]:
df_group = df.groupby(['Jogar', 'Umidade'])['Quantidade'].sum().reset_index()

# CALCULA AS PROBABILIDADES
UMIDADE_GAME, UMIDADE_NOT_GAME = calculateProbs('Umidade', df_group)
df_group

P(Jogo|Alta) = 0.3333333333333333
P(Not_Jogo|Alta) = 0.8

P(Jogo|Normal) = 0.6666666666666666
P(Not_Jogo|Normal) = 0.2



,Jogar,Umidade,Quantidade
0,Nao,Alta,4
1,Nao,Normal,1
2,Sim,Alta,3
3,Sim,Normal,6


#### 5. Vento 

In [113]:
df_group = df.groupby(['Jogar', 'Vento'])['Quantidade'].sum().reset_index()

# CALCULA AS PROBABILIDADES
VENTO_GAME, VENTO_NOT_GAME = calculateProbs('Vento', df_group)
df_group

P(Jogo|Forte) = 0.3333333333333333
P(Not_Jogo|Forte) = 0.6

P(Jogo|Fraco) = 0.6666666666666666
P(Not_Jogo|Fraco) = 0.4



,Jogar,Vento,Quantidade
0,Nao,Forte,3
1,Nao,Fraco,2
2,Sim,Forte,3
3,Sim,Fraco,6


### Classificador

A partir dos argumentos passados (condições de tempo), calcula se haverá partida ou não

In [114]:
def classifier(arguments):

    prob_play_golf = P_GAME * TEMPO_GAME[arguments[0]] * TEMPERATURA_GAME[arguments[1]] * UMIDADE_GAME[arguments[2]] * VENTO_GAME[arguments[3]]
    prob_not_play_golf = P_NOT_GAME * TEMPO_NOT_GAME[arguments[0]] * TEMPERATURA_NOT_GAME[arguments[1]] * UMIDADE_NOT_GAME[arguments[2]] * VENTO_NOT_GAME[arguments[3]]

    # Calcula as probabilidades reais
    real_prob_play_golf = prob_play_golf/(prob_play_golf + prob_not_play_golf)
    real_prob_not_play_golf = 1 - real_prob_play_golf

    awnser = "PLAY GOLF" if real_prob_play_golf > real_prob_not_play_golf else "NOT PLAY GOLF"

    return awnser, real_prob_play_golf, real_prob_not_play_golf

### Realizando Testes

In [118]:
# CÉLULA PARA DEFINIR OS PARÂMETROS

"""
TEMPO

0 : CHUVOSO
1 : ENSOLARADO
2 : NUBLADO
"""
tempo = 0

"""
TEMPERATURA

0 : AMENO
1 : FRIO
2 : QUENTE
"""
temperatura = 2

"""
UMIDADE

0 : ALTA
1 : NORMAL
"""
umidade = 1

"""
VENTO

0 : FORTE
1 : FRACO
"""
vento = 1

In [121]:
# Monta lista de argumentos e realiza a classificação

arguments = [tempo, temperatura, umidade, vento]

awnser, play_prob, not_play_prob = classifier(arguments)

print(f'Reposta final: {awnser}\nProbabilidade de jogar Golf = {play_prob:.2f}\nProbabilidade de não jogar Golf = {not_play_prob:.2f}')

Reposta final: PLAY GOLF
Probabilidade de jogar Golf = 0.74
Probabilidade de não jogar Golf = 0.26
